# Predicting target in train

### Notebook automatically generated from your model

Model XGBoost, trained on 2019-03-19 15:22:58.

#### Generated on 2019-03-19 15:37:00.004461

prediction
This notebook will reproduce the steps for a BINARY_CLASSIFICATION on  train.
The main objective is to predict the variable target

#### Warning

The goal of this notebook is to provide an easily readable and explainable code that reproduces the main steps
of training the model. It is not complete: some of the preprocessing done by the DSS visual machine learning is not
replicated in this notebook. This notebook will not give the same results and model performance as the DSS visual machine
learning model.

Let's start with importing the required libs :

In [1]:
import sys
import numpy as np
import pandas as pd
import sklearn as sk


And tune pandas display options:

#### Importing base data

The first step is to get our machine learning dataset:

In [2]:
# We apply the preparation that you defined. You should not modify this.

train = pd.read_csv("train.csv")
#test = pd.read_csv("test.csv")

train.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.100490,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,...,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537
std,0.300653,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,...,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,...,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800
25%,0.000000,8.453850,-4.740025,8.722475,5.254075,9.883175,-11.200350,4.767700,13.943800,-2.317800,...,-0.058825,5.157400,0.889775,0.584600,15.629800,-1.170700,-1.946925,8.252800,13.829700,-11.208475
50%,0.000000,10.524750,-1.608050,10.580000,6.825000,11.108250,-4.833150,5.385100,16.456800,0.393700,...,3.203600,7.347750,1.901300,3.396350,17.957950,-0.172700,2.408900,8.888200,15.934050,-2.819550
75%,0.000000,12.758200,1.358625,12.516700,8.324100,12.261125,0.924800,6.003000,19.102900,2.937900,...,6.406200,9.512525,2.949500,6.205800,20.396525,0.829600,6.556725,9.593300,18.064725,4.836800
max,1.000000,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.447700,27.691800,10.151300,...,18.440900,16.716500,8.402400,18.281800,27.928800,4.272900,18.321500,12.000400,26.079100,28.500700


In [3]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold

#### Initial data management

The preprocessing aims at making the dataset compatible with modeling.
At the end of this step, we will have a matrix of float numbers, with no missing values.
We'll use the features and the preprocessing steps defined in Models.

Let's only keep selected features

Let's first coerce categorical columns into unicode, numerical features into floats.

In [4]:
def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

We are now going to handle the target variable and store it in a new variable:

#### Cross-validation strategy

This is a simple cross-validation strategy.

#### Features preprocessing

We can now handle the categorical features (still using the settings defined in Models):

Let's rescale numerical features

In [5]:
rescale_features = {u'var_48': u'AVGSTD', u'var_49': u'AVGSTD', u'var_44': u'AVGSTD', u'var_45': u'AVGSTD', u'var_46': u'AVGSTD', u'var_47': u'AVGSTD', u'var_40': u'AVGSTD', u'var_41': u'AVGSTD', u'var_42': u'AVGSTD', u'var_43': u'AVGSTD', u'var_188': u'AVGSTD', u'var_189': u'AVGSTD', u'var_184': u'AVGSTD', u'var_185': u'AVGSTD', u'var_186': u'AVGSTD', u'var_187': u'AVGSTD', u'var_180': u'AVGSTD', u'var_181': u'AVGSTD', u'var_182': u'AVGSTD', u'var_183': u'AVGSTD', u'var_104': u'AVGSTD', u'var_105': u'AVGSTD', u'var_106': u'AVGSTD', u'var_107': u'AVGSTD', u'var_100': u'AVGSTD', u'var_101': u'AVGSTD', u'var_102': u'AVGSTD', u'var_103': u'AVGSTD', u'var_7': u'AVGSTD', u'var_6': u'AVGSTD', u'var_5': u'AVGSTD', u'var_4': u'AVGSTD', u'var_108': u'AVGSTD', u'var_109': u'AVGSTD', u'var_1': u'AVGSTD', u'var_0': u'AVGSTD', u'var_59': u'AVGSTD', u'var_58': u'AVGSTD', u'var_53': u'AVGSTD', u'var_52': u'AVGSTD', u'var_51': u'AVGSTD', u'var_50': u'AVGSTD', u'var_57': u'AVGSTD', u'var_56': u'AVGSTD', u'var_55': u'AVGSTD', u'var_54': u'AVGSTD', u'var_173': u'AVGSTD', u'var_172': u'AVGSTD', u'var_131': u'AVGSTD', u'var_130': u'AVGSTD', u'var_133': u'AVGSTD', u'var_132': u'AVGSTD', u'var_135': u'AVGSTD', u'var_134': u'AVGSTD', u'var_137': u'AVGSTD', u'var_136': u'AVGSTD', u'var_139': u'AVGSTD', u'var_138': u'AVGSTD', u'var_3': u'AVGSTD', u'var_179': u'AVGSTD', u'var_178': u'AVGSTD', u'var_28': u'AVGSTD', u'var_29': u'AVGSTD', u'var_26': u'AVGSTD', u'var_27': u'AVGSTD', u'var_24': u'AVGSTD', u'var_25': u'AVGSTD', u'var_22': u'AVGSTD', u'var_23': u'AVGSTD', u'var_20': u'AVGSTD', u'var_21': u'AVGSTD', u'var_128': u'AVGSTD', u'var_129': u'AVGSTD', u'var_126': u'AVGSTD', u'var_127': u'AVGSTD', u'var_124': u'AVGSTD', u'var_125': u'AVGSTD', u'var_122': u'AVGSTD', u'var_123': u'AVGSTD', u'var_120': u'AVGSTD', u'var_121': u'AVGSTD', u'var_39': u'AVGSTD', u'var_38': u'AVGSTD', u'var_35': u'AVGSTD', u'var_34': u'AVGSTD', u'var_37': u'AVGSTD', u'var_36': u'AVGSTD', u'var_31': u'AVGSTD', u'var_30': u'AVGSTD', u'var_33': u'AVGSTD', u'var_32': u'AVGSTD', u'var_159': u'AVGSTD', u'var_158': u'AVGSTD', u'var_153': u'AVGSTD', u'var_152': u'AVGSTD', u'var_151': u'AVGSTD', u'var_150': u'AVGSTD', u'var_157': u'AVGSTD', u'var_156': u'AVGSTD', u'var_155': u'AVGSTD', u'var_154': u'AVGSTD', u'var_80': u'AVGSTD', u'var_81': u'AVGSTD', u'var_82': u'AVGSTD', u'var_83': u'AVGSTD', u'var_84': u'AVGSTD', u'var_85': u'AVGSTD', u'var_86': u'AVGSTD', u'var_87': u'AVGSTD', u'var_88': u'AVGSTD', u'var_89': u'AVGSTD', u'var_148': u'AVGSTD', u'var_149': u'AVGSTD', u'var_140': u'AVGSTD', u'var_141': u'AVGSTD', u'var_142': u'AVGSTD', u'var_143': u'AVGSTD', u'var_144': u'AVGSTD', u'var_145': u'AVGSTD', u'var_146': u'AVGSTD', u'var_147': u'AVGSTD', u'var_17': u'AVGSTD', u'var_16': u'AVGSTD', u'var_15': u'AVGSTD', u'var_14': u'AVGSTD', u'var_13': u'AVGSTD', u'var_12': u'AVGSTD', u'var_11': u'AVGSTD', u'var_10': u'AVGSTD', u'var_8': u'AVGSTD', u'var_19': u'AVGSTD', u'var_18': u'AVGSTD', u'var_175': u'AVGSTD', u'var_174': u'AVGSTD', u'var_177': u'AVGSTD', u'var_176': u'AVGSTD', u'var_171': u'AVGSTD', u'var_170': u'AVGSTD', u'var_99': u'AVGSTD', u'var_98': u'AVGSTD', u'var_97': u'AVGSTD', u'var_96': u'AVGSTD', u'var_95': u'AVGSTD', u'var_94': u'AVGSTD', u'var_93': u'AVGSTD', u'var_92': u'AVGSTD', u'var_91': u'AVGSTD', u'var_90': u'AVGSTD', u'var_62': u'AVGSTD', u'var_63': u'AVGSTD', u'var_60': u'AVGSTD', u'var_61': u'AVGSTD', u'var_66': u'AVGSTD', u'var_67': u'AVGSTD', u'var_64': u'AVGSTD', u'var_65': u'AVGSTD', u'var_68': u'AVGSTD', u'var_69': u'AVGSTD', u'var_2': u'AVGSTD', u'var_162': u'AVGSTD', u'var_163': u'AVGSTD', u'var_160': u'AVGSTD', u'var_161': u'AVGSTD', u'var_166': u'AVGSTD', u'var_167': u'AVGSTD', u'var_164': u'AVGSTD', u'var_165': u'AVGSTD', u'var_168': u'AVGSTD', u'var_169': u'AVGSTD', u'var_71': u'AVGSTD', u'var_70': u'AVGSTD', u'var_73': u'AVGSTD', u'var_72': u'AVGSTD', u'var_75': u'AVGSTD', u'var_74': u'AVGSTD', u'var_77': u'AVGSTD', u'var_76': u'AVGSTD', u'var_79': u'AVGSTD', u'var_78': u'AVGSTD', u'var_9': u'AVGSTD', u'var_199': u'AVGSTD', u'var_198': u'AVGSTD', u'var_197': u'AVGSTD', u'var_196': u'AVGSTD', u'var_195': u'AVGSTD', u'var_194': u'AVGSTD', u'var_193': u'AVGSTD', u'var_192': u'AVGSTD', u'var_191': u'AVGSTD', u'var_190': u'AVGSTD', u'var_117': u'AVGSTD', u'var_116': u'AVGSTD', u'var_115': u'AVGSTD', u'var_114': u'AVGSTD', u'var_113': u'AVGSTD', u'var_112': u'AVGSTD', u'var_111': u'AVGSTD', u'var_110': u'AVGSTD', u'var_119': u'AVGSTD', u'var_118': u'AVGSTD'}
for (feature_name, rescale_method) in rescale_features.items():
    if rescale_method == 'MINMAX':
        _min = train[feature_name].min()
        _max = train[feature_name].max()
        scale = _max - _min
        shift = _min
    else:
        shift = train[feature_name].mean()
        scale = train[feature_name].std()
    if scale == 0.:
        del train[feature_name]
        del test[feature_name]
        print ('Feature %s was dropped because it has no variance' % feature_name)
    else:
        print ('Rescaled %s' % feature_name)
        train[feature_name] = (train[feature_name] - shift).astype(np.float64) / scale

Rescaled var_48
Rescaled var_49
Rescaled var_44
Rescaled var_45
Rescaled var_46
Rescaled var_47
Rescaled var_40
Rescaled var_41
Rescaled var_42
Rescaled var_43
Rescaled var_188
Rescaled var_189
Rescaled var_184
Rescaled var_185
Rescaled var_186
Rescaled var_187
Rescaled var_180
Rescaled var_181
Rescaled var_182
Rescaled var_183
Rescaled var_104
Rescaled var_105
Rescaled var_106
Rescaled var_107
Rescaled var_100
Rescaled var_101
Rescaled var_102
Rescaled var_103
Rescaled var_7
Rescaled var_6
Rescaled var_5
Rescaled var_4
Rescaled var_108
Rescaled var_109
Rescaled var_1
Rescaled var_0
Rescaled var_59
Rescaled var_58
Rescaled var_53
Rescaled var_52
Rescaled var_51
Rescaled var_50
Rescaled var_57
Rescaled var_56
Rescaled var_55
Rescaled var_54
Rescaled var_173
Rescaled var_172
Rescaled var_131
Rescaled var_130
Rescaled var_133
Rescaled var_132
Rescaled var_135
Rescaled var_134
Rescaled var_137
Rescaled var_136
Rescaled var_139
Rescaled var_138
Rescaled var_3
Rescaled var_179
Rescaled var_1

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,-0.577100,-1.273734,0.451706,-0.833707,0.235571,-0.536429,-0.334925,0.608749,...,0.263373,-1.149155,0.817467,-0.411012,0.168704,-1.578113,1.022128,-0.373967,-1.026395,0.214134
1,train_1,0,0.269958,-0.622136,1.190357,-0.688845,0.790973,1.539897,0.244461,-0.003525,...,0.966609,0.093604,0.443622,1.908760,-0.817592,1.522338,1.067651,-0.129399,0.825415,0.505684
2,train_2,0,-0.681112,-0.276066,0.516987,0.536515,-0.305476,-0.511032,1.769834,-0.564748,...,-0.072093,0.777995,-0.174130,-0.412315,1.151588,2.297364,-1.617902,-0.695139,-0.381448,0.356681
3,train_3,0,0.125158,-0.129425,-0.667574,0.195354,0.927990,0.410671,0.500632,-0.474200,...,0.270215,-0.891454,-0.818466,-0.478547,1.607865,-0.789515,-0.959017,1.501741,0.697116,-0.543500
4,train_4,0,-0.277303,0.035610,0.817681,-0.077829,0.738605,0.955572,0.613370,0.791542,...,-1.036189,0.688986,-1.405984,1.468532,-1.501098,-0.958471,0.297626,0.645536,0.706316,-0.525374


In [14]:
from sklearn import preprocessing
features = [c for c in train.columns if c not in ['ID_code', 'target']]

#X_test = test[features].values
#X = train.drop(['target','ID_code'], axis=1)
#y = np.array(train['target'])


#   
#normalized_X = preprocessing.normalize([x_array])

array([[-0.04222029, -0.09318556,  0.03304654, ..., -0.0273592 ,
        -0.07509043,  0.01566593],
       [ 0.01812239, -0.04176426,  0.07990922, ..., -0.00868663,
         0.05541048,  0.03394679],
       [-0.05366426, -0.021751  ,  0.040733  , ..., -0.05476944,
        -0.03005402,  0.02810258],
       ...,
       [ 0.01325155, -0.06269251, -0.00568559, ..., -0.01550069,
        -0.05018594,  0.05346474],
       [-0.02109078, -0.11453175,  0.07283222, ...,  0.08116989,
        -0.00754205, -0.06729738],
       [ 0.0049826 , -0.07779551,  0.04100039, ..., -0.0604931 ,
        -0.09591487,  0.02561756]])

In [18]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.335,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.041,
    'learning_rate': 0.0083,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary',
    'verbosity': 1}

In [22]:
num_folds = 2
target = train['target']
features = [c for c in train.columns if c not in ['ID_code', 'target']]
normalizer = preprocessing.Normalizer().fit(train[features])  # fit does nothing
normalizer
normalizer.transform(train[features])  

folds = KFold(n_splits=num_folds, random_state=2319)
oof = np.zeros(len(train))
getVal = np.zeros(len(train))
predictions = np.zeros(len(target))
feature_importance_df = pd.DataFrame()

In [24]:


print('Light GBM Model')
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    
    X_train, y_train = train.iloc[trn_idx][features], target.iloc[trn_idx]
    X_valid, y_valid = train.iloc[val_idx][features], target.iloc[val_idx]
    
    X_tr, y_tr = augment(X_train.values, y_train.values)
    X_tr = pd.DataFrame(X_tr)
    
    print("Fold:{}".format(fold_ + 1))
    trn_data = lgb.Dataset(X_tr, label=y_tr)
    val_data = lgb.Dataset(X_valid, label=y_valid)
    val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx])
    
    clf = lgb.train(param, trn_data, 1000000, valid_sets = [trn_data, val_data], verbose_eval=5000, early_stopping_rounds = 4000)
    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    getVal[val_idx]+= clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration) / folds.n_splits
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    


Light GBM Model
Fold:1
Training until validation scores don't improve for 4000 rounds.
[5000]	training's auc: 0.921814	valid_1's auc: 0.894835
[10000]	training's auc: 0.935552	valid_1's auc: 0.89705
Early stopping, best iteration is:
[9499]	training's auc: 0.934328	valid_1's auc: 0.897143


NameError: name 'test' is not defined

In [26]:
test = pd.read_csv("test.csv")
normalizer.transform(test[features])  
predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

CV score: 0.59805 


In [33]:
Pred2=pd.DataFrame(predictions)
Pred2['target'] = np.mean(Pred2[[col for col in Pred2.columns if col not in ['ID_code', 'target']]].values, axis=1)
sub = pd.DataFrame({"ID_code": test.ID_code.values})
#sub["target"] = predictions['target']
#sub.to_csv('submission_PE.csv', index=False)

That's it. It's now up to you to tune your preprocessing, your algo, and your analysis !
